#These are all the Python libraries needed:

geopandas for handling CAFO polygons.

rasterio for working with raster/NAIP tiles.

pystac_client and planetary_computer to query and sign NAIP tiles.

Path and typing.List for path management and type hints.

In [ ]:
import traceback
from pathlib import Path
from typing import List
import geopandas as gpd
import rasterio
from rasterio.windows import from_bounds
from shapely.geometry import shape
from pystac_client import Client
from planetary_computer import sign


Set your workspace folder and all paths.

TILE_DIR: where NAIP tiles will be saved.

INDEX_PATH: saves a GeoJSON index of all tiles.

CAFO_LABELS_PATH: your CAFO polygons file.

mkdir ensures the directories exist before saving tiles.

In [ ]:
# ==== CONFIGURATION ====
START_TILE = 3
END_TILE = 4
BBOX = [-77.5, 36.8, -75.0, 39.8]  # Chesapeake Bay bounding box
YEARS = "2018-01-01/2018-12-31"    # NAIP tiles time range

# ==== PATHS ====
BASE_PATH = Path("/content/cafo_project")  # Workspace folder
TILE_DIR = BASE_PATH / "data/tiles_2018"   # Downloaded tiles
INDEX_PATH = BASE_PATH / "data/naip_tile_index_2018.geojson"  # Tile index
CAFO_LABELS_PATH = BASE_PATH / "data/cafo_labels.geojson"      # CAFO polygons
TILE_DIR.mkdir(parents=True, exist_ok=True)


Loads CAFO polygons from your GeoJSON file into a GeoDataFrame.

In [ ]:
def query_naip_tiles(bbox: List[float], years: str) -> List:
    """Query NAIP tiles from Planetary Computer STAC API."""
    print("Querying NAIP tiles...")
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    items = list(catalog.search(collections=["naip"], bbox=bbox, datetime=years).get_items())
    print(f"Found {len(items)} NAIP tiles.")
    return items

# Query NAIP tiles
items = query_naip_tiles(BBOX, YEARS)


Connects to Planetary Computer STAC API.

Retrieves NAIP tiles in the specified bounding box and date range.

In [ ]:
def create_tiles_gdf(items: List) -> gpd.GeoDataFrame:
    """Convert STAC items to GeoDataFrame with signed URLs."""
    urls, geoms = [], []
    for item in items:
        s = sign(item)
        geoms.append(shape(s.geometry))
        urls.append(s.assets["image"].href)
    return gpd.GeoDataFrame({"tile_url": urls}, geometry=geoms, crs="EPSG:4326")

tiles_gdf = create_tiles_gdf(items)
tiles_gdf.to_file(INDEX_PATH)  # Save tile index


Keeps only tiles that intersect with CAFO polygons.

Limits selection for testing or batching with START_TILE and END_TILE.

In [ ]:
def download_tiles(urls: List[str], geoms: List, tile_dir: Path, start_idx: int):
    """Download selected NAIP tiles and crop to CAFO polygons."""
    for i, (url, geom) in enumerate(zip(urls, geoms), start=start_idx):
        try:
            signed_url = sign(url)
            with rasterio.open(signed_url) as src:
                geom_proj = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(src.crs)
                bounds = geom_proj.total_bounds
                window = from_bounds(*bounds, transform=src.transform)
                profile = src.profile.copy()
                profile.update({
                    "height": int(window.height),
                    "width": int(window.width),
                    "transform": rasterio.windows.transform(window, src.transform)
                })
                path = tile_dir / f"tile_{i}.tif"
                with rasterio.open(path, "w", **profile) as dst:
                    for b in range(1, src.count + 1):
                        dst.write(src.read(b, window=window), b)
            print(f"Saved tile {i} to {path}")
        except Exception as e:
            print(f"Failed tile {i}: {e}")
            traceback.print_exc()

# Download tiles
download_tiles(urls, geoms, TILE_DIR, START_TILE)
